In [1]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los CSVs
ruta_carpeta = './'

# Cargar todos los archivos CSV
dfs = []
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
        df['Fuente'] = archivo  # Agregamos una columna para identificar el origen
        dfs.append(df)

# Combinar todos los DataFrames en uno solo
df_todos = pd.concat(dfs, ignore_index=True)


In [3]:
import pandas as pd
import os
import numpy as np

# Definición de la lista de métricas deseadas
METRICAS = ['SMAPE', 'MAE', 'MSE']

def load_results(filename):
    """
    Carga el CSV y extrae el número de intervalo a partir de la columna 'Intervalos Acumulados'.
    Se asume que la cadena tiene el formato 'Intervalos 1 a X'.
    """
    df = pd.read_csv(filename)
    # Extraemos el número final del intervalo para usarlo como índice
    df['Intervalo'] = df['Intervalos Acumulados'].str.extract(r'a (\d+)').astype(int)
    df.set_index('Intervalo', inplace=True)
    return df

def extraer_metricas(df, metricas=METRICAS):
    """
    Asegura que el DataFrame tenga las columnas indicadas en 'metricas'.
    Si falta alguna, se agrega con valores NaN.
    Se reordena el DataFrame según el orden especificado en 'metricas'.
    """
    for met in metricas:
        if met not in df.columns:
            df[met] = np.nan
    return df[metricas]

def obtener_tabla(horizonte):
    # Diccionario para almacenar DataFrames para cada modelo
    tablas = {}

    # Lista de métricas a extraer
    metricas = METRICAS
    
    # Archivos para los modelos LSTM
    lstm_files = {
        'LSTM-7': f"lstm_7_{horizonte}.csv",
        'LSTM-14': f"lstm_14_{horizonte}.csv",
        'LSTM-21': f"lstm_21_{horizonte}.csv"
    }
    
    # Procesar archivos de LSTM
    for key, file in lstm_files.items():
        if os.path.exists(file):
            df = load_results(file)
            # Extraer las métricas, agregando NaN si no existen (por ejemplo, MSE)
            tablas[key] = extraer_metricas(df, metricas)
        else:
            raise FileNotFoundError(f"Archivo no encontrado: {file}")
    
    # Procesar archivo de TimesFM
    timesfm_file = f"timesfm_{horizonte}.csv"
    if not os.path.exists(timesfm_file):
        raise FileNotFoundError(f"Archivo no encontrado: {timesfm_file}")
    df_timesfm = load_results(timesfm_file)
    tablas['TimesFM'] = extraer_metricas(df_timesfm, metricas)
    
    # Combinar todos los DataFrames en uno con columnas de MultiIndex.
    # El primer nivel corresponde al modelo y el segundo nivel a la métrica.
    tabla_resumen = pd.concat(tablas, axis=1)
    tabla_resumen.index.name = 'Intervalo'
    
    return tabla_resumen

# Obtener y mostrar la tabla para horizonte 15
tabla_15 = obtener_tabla(15)
print("Tabla para horizonte 15:")
print(tabla_15)
tabla_15.to_csv('tabla_horizonte_15.csv')

# Obtener y mostrar la tabla para horizonte 30
tabla_30 = obtener_tabla(30)
print("\nTabla para horizonte 30:")
print(tabla_30)
tabla_30.to_csv('tabla_horizonte_30.csv')


Tabla para horizonte 15:
               LSTM-7                                LSTM-14              \
                SMAPE         MAE            MSE       SMAPE         MAE   
Intervalo                                                                  
1           28.994249  137.059725   35813.055372   16.716748   63.385491   
2           19.427937   98.813068   13173.480206   20.486859  103.818899   
3           28.384518  265.685872  120316.701522   28.233642  263.162195   
4           50.134744   13.459221     218.092921   34.324984    6.924997   
5           48.055877  118.838378   19506.889712   26.462351   53.623151   
6           14.375691   18.198282     672.365458   13.297392   16.982417   
7          105.960527   21.556710     787.111525   49.063209   14.252785   
8           94.609504   10.278951     113.401167  127.046611   20.643033   

                            LSTM-21                                TimesFM  \
                     MSE      SMAPE         MAE            M